## Local Inference on GPU 
Model page: https://huggingface.co/saadkhi/SQL_Chat_finetuned_model

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/saadkhi/SQL_Chat_finetuned_model)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [1]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 32.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 24.7 MB/s eta 0:00:0000:01:00:01
  Attempting un

In [2]:
from peft import PeftModel
from transformers import AutoModelForCausalLM

base_model = AutoModelForCausalLM.from_pretrained("unsloth/Phi-3-mini-4k-instruct-bnb-4bit")
model = PeftModel.from_pretrained(base_model, "saadkhi/SQL_Chat_finetuned_model")

2025-12-01 14:19:10.963317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764598751.163399      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764598751.224919      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/478M [00:00<?, ?B/s]

In [3]:
# FULLY WORKING CHAT WITH YOUR MODEL (Phi-3 Mini + Your LoRA)

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# 1. Load the base 4bit model (auto uses bitsandbytes)
base_model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"
print("Loading base model (this takes ~20–40 seconds first time)...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    load_in_4bit=True,           # Keeps it fast and low VRAM (~6–8GB)
)

# 2. Load your LoRA adapter on top
print("Loading your fine-tuned LoRA...")
model = PeftModel.from_pretrained(base_model, "saadkhi/SQL_Chat_finetuned_model")

# 3. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)

# 4. Enable fast inference mode (optional but recommended)
model.eval()

print("\nModel loaded! Start chatting (type 'quit' to exit)\n")

# 5. Interactive chat loop
while True:
    user_input = input("You: ").strip()
    if user_input.lower() in ["quit", "exit", "bye"]:
        print("Goodbye!")
        break
    if not user_input:
        continue

    # Format exactly like Phi-3 expects
    messages = [{"role": "user", "content": user_input}]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        inputs,
        max_new_tokens=512,
        temperature=0.8,
        do_sample=True,
        top_p=0.9,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    # Clean up the output (remove input part + special tokens)
    response = response.split("<|assistant|>")[-1].split("<|end|>")[0].strip()

    print(f"\nModel: {response}\n")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading base model (this takes ~20–40 seconds first time)...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading your fine-tuned LoRA...


tokenizer_config.json: 0.00B [00:00, ?B/s]

...


Model: SELECT (SUM(CASE WHEN EXTRACT(MONTH FROM o2.order_date) = 1 THEN o2.amount ELSE 0 END) - SUM(CASE WHEN EXTRACT(MONTH FROM o1.order_date) = 11 THEN o1.amount ELSE 0 END)) / SUM(CASE WHEN EXTRACT(MONTH FROM o1.order_date) = 11 THEN o1.amount ELSE 0 END) * 100 AS mom_growth FROM orders o1 JOIN orders o2 ON EXTRACT(YEAR FROM o2.order_date) = EXTRACT(YEAR FROM o1.order_date) + 1;



You:  sql bot



Model: Sorry, I can only answer questions related to SQL and NoSQL databases.<|endoftext|>



You:  quit


Goodbye!
